In [53]:
import csv
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
import warnings
import gensim
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd

In [54]:
with open('../dataset/dataset_generated.txt') as f:
    lines = f.readlines()

In [55]:
print(len(lines))

324778


In [56]:
lines[4]

'the author that the guard likes smiles . <eos>\n'

In [57]:
x = "no authors that the guards like have ever been popular . <eos>\n"

In [58]:
k = lines.index(x)

In [59]:
sentences1 = []
judgement1 = []
for i in range(len(lines[:k])):
    line = lines[i].split(".")
    sent = line[0]
    sentences1.append(sent)
    correct = 0
    if (i % 2) == 0:
        correct = 1
    judgement1.append(correct)

In [60]:
df1 = pd.DataFrame(list(zip(sentences1, judgement1)), columns =['Sentence', 'Judgement'])

In [61]:
df1

,Sentence,Judgement
0,the author that the guard likes laughs,1
1,the author that the guard likes laugh,0
2,the author that the guard likes swims,1
3,the author that the guard likes swim,0
4,the author that the guard likes smiles,1
...,...,...
294515,the bankers knew the consultants hated herself,0
294516,the bankers knew the consultants doubted thems...,1
294517,the bankers knew the consultants doubted himself,0
294518,the bankers knew the consultants doubted thems...,1


In [62]:
sentences2 = []
judgement2 = []
for i in range(len(lines[k:])):
    j = k + i
    line = lines[j].split(".")
    sent = line[0]
    sentences2.append(sent)
    correct = 0
    if (j % 3) == 1:
        correct = 1
    judgement2.append(correct)

In [63]:
df2 = pd.DataFrame(list(zip(sentences2, judgement2)), columns =['Sentence', 'Judgement'])

In [64]:
df2

,Sentence,Judgement
0,no authors that the guards like have ever been...,1
1,the authors that no guards like have ever been...,0
2,most authors that the guards like have ever be...,0
3,no authors that the guards like have ever been...,1
4,the authors that no guards like have ever been...,0
...,...,...
30253,some shows will ever be ignored,0
30254,many shows will ever be ignored,0
30255,few shows will ever get old,1
30256,some shows will ever get old,0


In [69]:
list_sent = sentences1 + sentences2

In [70]:
len(list_sent)

324778

In [72]:
list_sent_tkn = []
for sent in list_sent:
    list_sent_tkn.append(word_tokenize(sent))

In [77]:
len(list_sent_tkn)

324778

In [74]:
list_sent_tkn[2]

['the', 'author', 'that', 'the', 'guard', 'likes', 'swims']

In [83]:
w2v_model = Word2Vec(list_sent_tkn, min_count=1, vector_size= 50, workers=3, window=3)

In [84]:
words = list(w2v_model.wv.index_to_key)
print(len(words))

162


In [85]:
words

['the',
 'that',
 'are',
 'is',
 'like',
 'admire',
 'love',
 'hate',
 'ever',
 'likes',
 'hates',
 'admires',
 'loves',
 'taxi',
 'old',
 'themselves',
 'people',
 'to',
 'young',
 'tall',
 'short',
 'popular',
 'executives',
 'drivers',
 'dancers',
 'guards',
 'ministers',
 'chefs',
 'architects',
 'skaters',
 'parents',
 'assistants',
 'have',
 'will',
 'skater',
 'parent',
 'executive',
 'assistant',
 'driver',
 'dancer',
 'minister',
 'architect',
 'chef',
 'guard',
 'herself',
 'himself',
 'unpopular',
 'joy',
 'new',
 'bad',
 'good',
 'teachers',
 'consultants',
 'senators',
 'managers',
 'officers',
 'customers',
 'farmers',
 'surgeons',
 'pilots',
 'authors',
 'be',
 'of',
 'laughs',
 'smiles',
 'laugh',
 'swims',
 'swim',
 'smile',
 'from',
 'author',
 'teacher',
 'surgeon',
 'pilot',
 'senator',
 'officer',
 'customer',
 'manager',
 'farmer',
 'consultant',
 'no',
 'few',
 'been',
 'hated',
 'doubted',
 'embarrassed',
 'congratulated',
 'injured',
 'disguised',
 'hurt',
 'sh

In [86]:
word2vec_model = KeyedVectors.load_word2vec_format('embedding.txt', binary=False)

In [87]:
words = list(word2vec_model.index_to_key)
print(len(words))

14532


In [88]:
word_vec_dict={}
for word in words:
    word_vec_dict[word]=word2vec_model.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size

The no of key-value pairs :  14532


In [89]:
w2v_weights = word2vec_model.vectors
vocab_size, embedding_size = w2v_weights.shape
print(vocab_size, embedding_size)
MAX_SEQUENCE_LENGTH = 200

14532 300


In [90]:
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant
import gensim
import sys
import numpy as np
import pandas as pd
import string
import logging
import random
import keras
import matplotlib.pyplot as plt

In [91]:
df = pd.concat([df1, df2])

In [92]:
df

,Sentence,Judgement
0,the author that the guard likes laughs,1
1,the author that the guard likes laugh,0
2,the author that the guard likes swims,1
3,the author that the guard likes swim,0
4,the author that the guard likes smiles,1
...,...,...
30253,some shows will ever be ignored,0
30254,many shows will ever be ignored,0
30255,few shows will ever get old,1
30256,some shows will ever get old,0


In [95]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


maxi=-1
for i,rev in enumerate(df['Sentence']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)


tok = Tokenizer(filters='')
tok.fit_on_texts(df['Sentence'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['Sentence'])

print(vocab_size)

163


In [96]:
pad_rev= pad_sequences(encd_rev, maxlen=maxi+1, padding='post')
pad_rev.shape   # note that we had 100K reviews and we have padded each review to have  a lenght of 1565 words.
print(pad_rev)
print(pad_rev.shape)

[[  1  71   2 ...   0   0   0]
 [  1  71   2 ...   0   0   0]
 [  1  71   2 ...   0   0   0]
 ...
 [ 82  91  34 ...   0   0   0]
 [162  91  34 ...   0   0   0]
 [111  91  34 ...   0   0   0]]
(324778, 15)


In [97]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embedding_size))
# print(len(tok.word_index.items()))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

train_size=0.8

y = np.array(df['Judgement'].tolist())

leinst = LabelEncoder()
yinst = leinst.fit_transform(y)

ycat = to_categorical(df['Judgement'])  # one hot target as required by NN
# print(ycat)
# X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_train, X_test, y_train, y_test = train_test_split(pad_rev, ycat, test_size=0.2, shuffle=True, random_state=42)

# test_size = 0.5
# X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=42)


print(X_train.shape), print(y_train.shape)
print(X_validation.shape), print(y_validation.shape)
print(X_test.shape), print(y_test.shape)

(259822, 15)
(259822, 2)
(32478, 15)
(32478, 2)
(32478, 15)
(32478, 2)


(None, None)

In [98]:
num_classes = len(leinst.classes_)
print(num_classes)
print(leinst.classes_)

2
[0 1]


In [99]:
print(w2v_weights.shape)

(14532, 300)


In [108]:
y_validation[20:50]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [109]:
model = Sequential()
# Keras Embedding layer with Word2Vec weights initialization
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    weights=[embed_matrix],
                    input_length=15,
                    ))

model.add(Bidirectional(LSTM(30)))
# model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
# model.add(Flatten())
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, batch_size=64,
                    validation_data=(X_validation, y_validation), verbose=1)

Epoch 1/5
4060/4060 [==============================] - 50s 12ms/step - loss: 0.0586 - accuracy: 0.9644 - val_loss: 8.7534e-05 - val_accuracy: 1.0000
Epoch 2/5
4060/4060 [==============================] - 50s 12ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 6.2334e-05 - val_accuracy: 1.0000
Epoch 3/5
4060/4060 [==============================] - 50s 12ms/step - loss: 2.6350e-05 - accuracy: 1.0000 - val_loss: 7.1157e-06 - val_accuracy: 1.0000
Epoch 4/5
4060/4060 [==============================] - 52s 13ms/step - loss: 2.8599e-06 - accuracy: 1.0000 - val_loss: 7.7508e-07 - val_accuracy: 1.0000
Epoch 5/5
4060/4060 [==============================] - 55s 14ms/step - loss: 3.2377e-07 - accuracy: 1.0000 - val_loss: 9.7935e-08 - val_accuracy: 1.0000


In [113]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 9.778363363466269e-08 / Test accuracy: 1.0


In [111]:
X_test

array([[  1,  52,   2, ...,   0,   0,   0],
       [  1,  61,   2, ...,   0,   0,   0],
       [  1, 121,   2, ...,   0,   0,   0],
       ...,
       [  1,  92,   2, ...,   0,   0,   0],
       [  1,  53,   2, ...,   0,   0,   0],
       [  1,  61, 106, ...,   0,   0,   0]])

In [114]:
predictions = model.predict([X_test])

In [115]:
predictions

array([[1.0000000e+00, 6.4676736e-17],
       [5.4803032e-15, 1.0000000e+00],
       [1.0000000e+00, 1.4890333e-14],
       ...,
       [1.0000000e+00, 1.0264168e-14],
       [2.6144365e-14, 1.0000000e+00],
       [1.0000000e+00, 2.6892684e-15]], dtype=float32)

In [116]:
n = random.randint(0, len(y_test))
i = np.argmax(predictions[n])
print("Prediction: ", leinst.classes_[i])

Prediction:  1


In [117]:
n

20222

In [118]:
X_test[n]

array([ 1, 71,  2,  1, 38, 13, 89, 46,  0,  0,  0,  0,  0,  0,  0])